# DAAI Project 3 - Train On GTA5 (step 2b)

## Data Preparation

### Clone the professor's repository

In [ ]:
!git clone https://github.com/ClaudiaCuttano/AML_Semantic_DA.git

### Mount Google Drive to access files

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

### Replace the empty 'cityscapes.py' with the one in this repo

In [ ]:
# when training with transforms copy GTA_transforms.py instead of GTA.py
!cp datasets/GTA.py AML_Semantic_DA/cityscapes.py

### Extract the GTA5 dataset

In [ ]:
import zipfile

with zipfile.ZipFile(f'drive/MyDrive/GTA5.zip', 'r') as zip_ref:
  zip_ref.extractall()

### Copy the pre-trained model to Colab

In [ ]:
!cp drive/MyDrive/STDCNet813M_73.91.tar STDCNet813M_73.91.tar

### Install tensorboardX

In [ ]:
!pip install tensorboardX

### Split the GTA5 dataset into train and val

In [ ]:
from sklearn.model_selection import train_test_split
import shutil
import os

test_size = 0.3
img_dir = 'GTA5/images'

all_files = os.listdir(img_dir)

# use random_state to always have the same images in train and val
train_files, test_files = train_test_split(all_files, test_size=test_size, random_state=10)

train_folder = os.path.join(img_dir, 'train')
test_folder = os.path.join(img_dir, 'val')

os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)
os.makedirs(train_folder.replace('images', 'labels'), exist_ok=True)
os.makedirs(test_folder.replace('images', 'labels'), exist_ok=True)

for img in train_files:
    source_path = os.path.join(img_dir, img)
    destination_path = os.path.join(train_folder, img)
    shutil.move(source_path, destination_path)
    shutil.move(source_path.replace('images', 'labels'), destination_path.replace('images', 'labels'))

for img in test_files:
    source_path = os.path.join(img_dir, img)
    destination_path = os.path.join(test_folder, img)
    shutil.move(source_path, destination_path)
    shutil.move(source_path.replace('images', 'labels'), destination_path.replace('images', 'labels'))

## Train the model

In [ ]:
!python AML_Semantic_DA/train.py \
--save_model_path drive/MyDrive/GTA5/ \
--backbone STDCNet813 \
--mode train \
--pretrain_path STDCNet813M_73.91.tar \
--num_workers 2 \
--num_epochs 20 \
--batch_size 2 \
--validation_step 5 \
--learning_rate 0.000302 \
--checkpoint_step 5

## Visualize The Results

### Create dataset and dataloader

In [ ]:
from pathlib import Path
from torch.utils.data import DataLoader
import os
import matplotlib.pyplot as plt
from AML_Semantic_DA.cityscapes import CityScapes


BATCH_SIZE = 1
NUM_WORKERS = os.cpu_count()
num_classes = 19

val_dataset = CityScapes(
    mode='val'
    )
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=True
)

### Load the trained model

In [ ]:
from AML_Semantic_DA.model.model_stages import BiSeNet
import torch

model = BiSeNet(backbone='STDCNet813', n_classes=num_classes, pretrain_model='STDCNet813M_73.91.tar', use_conv_last=False)
model.load_state_dict(torch.load('drive/MyDrive/GTA5/best.pth', map_location='cpu'))
model = torch.nn.DataParallel(model).cuda()

### Get an image and its label

In [ ]:
data, label = next(iter(val_loader))

### Generate prediction and calculate metrics

In [ ]:
import numpy as np
from AML_Semantic_DA.utils import poly_lr_scheduler
from AML_Semantic_DA.utils import reverse_one_hot, compute_global_accuracy, fast_hist, per_class_iu

with torch.no_grad():
  model.eval()
  hist = np.zeros((num_classes, num_classes))
  label = label.type(torch.LongTensor)
  data = data.cuda()
  label = label.long().cuda()

  # get RGB predict image
  predict, _, _ = model(data)
  predict = predict.squeeze(0)
  predict = reverse_one_hot(predict)
  predict = np.array(predict.cpu())

  # get RGB label image
  label = label.squeeze()
  label = np.array(label.cpu())

  # compute per pixel accuracy
  precision = compute_global_accuracy(predict, label)
  hist += fast_hist(label.flatten(), predict.flatten(), num_classes)

  precision = np.mean(precision_record)
  miou_list = per_class_iu(hist)
  miou = np.mean(miou_list)
  print('precision per pixel for test: %.3f' % precision)
  print('mIoU for validation: %.3f' % miou)
  print(f'mIoU per class: {miou_list}')

### Show the pictures

In [ ]:
plt.imshow(data[0].permute(1,2,0).cpu())

In [ ]:
label[label==255] = 0
plt.imshow(label)

In [ ]:
plt.imshow(predict)